# 1. Libraries

In [55]:
import numpy as np
import pandas as pd
import datetime
import os
from scipy.signal import find_peaks

# 2. Data load

In [56]:
df_nivel = pd.read_csv('/home/nivelrios/documentos/Mburicao Project/data/processed/nivel_processed.csv')

# 3. Peak detection

In [57]:
df_nivel['fecha'] = pd.to_datetime(df_nivel['fecha'])

UMBRAL = 0.3 

fechas = df_nivel['fecha'].values
niveles = df_nivel['nivel'].values

indices_inicio = np.where((niveles[:-1] < threshold) & (niveles[1:] >= threshold))[0] + 1
indices_fin = np.where((niveles[:-1] >= threshold) & (niveles[1:] < threshold))[0] + 1

eventos = []
for ini in indices_inicio:
    fines_posteriores = indices_fin[indices_fin > ini]
    if fines_posteriores.size > 0:
        fin = fines_posteriores[0]
        eventos.append((ini, fin))

eventos_list = []

for inicio, fin in eventos:
    fecha_inicio = fechas[inicio]
    fecha_fin = fechas[fin]
    
    duracion_evento = (fecha_fin - fecha_inicio) / np.timedelta64(1, 'm')
    
    segmento_niveles = niveles[inicio:fin]
    segmento_fechas = fechas[inicio:fin]
    
    picos_rel, _ = find_peaks(segmento_niveles)
    
    if len(picos_rel) > 0:
        idx_global_rel = np.argmax(segmento_niveles[picos_rel])
        pico_global_rel = picos_rel[idx_global_rel]
        
        pico_global_valor = segmento_niveles[pico_global_rel]
        pico_global_fecha = segmento_fechas[pico_global_rel]
        
        duracion_pico_global = (pico_global_fecha - fecha_inicio) / np.timedelta64(1, 'm')
        
        # ------------- PRIMER PICO LOCAL -------------#
        # Condiciones:
        # 1) Ocurra antes del pico global (fecha)
        # 2) Su valor >= threshold + 0.4
        # 3) Tomar el primero que cumpla estas condiciones
        pico_local_valor = None
        pico_local_fecha = None
        duracion_pico_local = None
        
        picos_ordenados = sorted(picos_rel)
        
        for p_rel in picos_ordenados:
            if p_rel < pico_global_rel:  
                valor_pico_local = segmento_niveles[p_rel]
                if valor_pico_local >= threshold + 0.4:
                    fecha_pico_local = segmento_fechas[p_rel]
                    pico_local_valor = valor_pico_local
                    pico_local_fecha = fecha_pico_local
                    duracion_pico_local = (pico_local_fecha - fecha_inicio) / np.timedelta64(1, 'm')
                    break  
        
    else:
        pico_global_valor = None
        pico_global_fecha = None
        duracion_pico_global = None
        pico_local_valor = None
        pico_local_fecha = None
        duracion_pico_local = None
    
    evento_info = {
        "inicio": fecha_inicio,
        "fin": fecha_fin,
        "evento_duracion": duracion_evento,
        "max_global": pico_global_valor,
        "max_global_fecha": pico_global_fecha,
        "max_global_duracion": duracion_pico_global,
        "max_local": pico_local_valor,
        "max_local_fecha": pico_local_fecha,
        "max_local_duracion": duracion_pico_local
    } 
    eventos_list.append(evento_info)

# 4. Save dataset

In [49]:
ruta = "/home/nivelrios/documentos/Mburicao Project/data/external"
os.makedirs(ruta, exist_ok=True)
archivo = os.path.join(ruta, "peak_nivel_detection.csv")
df_eventos.to_csv(archivo, index=False)